In [26]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [27]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [37]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True) if soup.body else "No body content"

    def __repr__(self):
        return (
            f"<Website(url='{self.url}', title='{self.title}', "
            f"text='{self.text[:200]}...')>"  # Show first 200 characters of text
        )

    def __str__(self):
        return (
            f"Website Analysis:\n"
            f"URL: {self.url}\n"
            f"Title: {self.title}\n"
            f"Content Preview:\n{self.text[:500]}..."  # Show first 500 characters
        )

In [38]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [39]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [40]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0")

In [41]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = llm.invoke(messages_for(website))
    return response.content




In [43]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [44]:
display_summary("https://www.ashokleyland.com/")

Website Analysis:
URL: https://www.ashokleyland.com/
Title: Ashok Leyland – Trucks, Buses, Defence, Light Vehicles and Power Solutions
Content Preview:
Trucks
ICV Trucks
ICV Trucks
View More
BOSS
Partner Super
ecomet Star
ICV Tippers
Haulage
Haulage
View More
6X4
10X2 (GVW : 48T)
8X2 (GVW : 35T)
6X2 (GVW : 26T - 31T)
4x2 (GVW : 19T)
Tippers
Tippers
View More
Transit Mixer 8X4
10x2 DTLA
10X4 DTLA
10X4
10X2 (GVW : 42T)
Transit Mixer 6X4
8X4 (GVW : 35T)
8X2 (GVW : 35T)
Tractors
Tractors
View More
6X4 with 3-axle Trailer
4X2 with 3-axle Trailer
4X2 with 2-axle Trailer
Buses
Brands
Brands
View More
GARUD 12M
GARUD 13.5M
Viking
Cheetah
Sunshine
Oyste...


Here's a short summary of the Ashok Leyland website in markdown:

# Ashok Leyland - Vehicle Manufacturer

Ashok Leyland is a diversified vehicle manufacturer offering a wide range of products across several categories:

## Main Product Lines

1. **Trucks**: Including ICV trucks, haulage vehicles, tippers, and tractors
2. **Buses**: Various models like GARUD, Viking, Cheetah, and Sunshine
3. **Light Vehicles**: Small commercial vehicles (SCV) and light commercial vehicles (LCV) for goods and passenger transport
4. **Power Solutions**: Diesel gensets, industrial and agricultural engines, and marine engines
5. **Defence**: Armored vehicles, high mobility vehicles, and specialized military logistics vehicles

## Key Features

- Extensive product range catering to different commercial and defense needs
- Manufacturing presence in 5 countries with 10 plants
- Production capacity of 500+ vehicles per day
- Focus on innovation and sustainability
- Offers aftermarket services, telematics solutions, and vehicle financing options

## Company Values

- Innovation
- Customer Centricity
- Trust
- Partnership
- Agility

The website showcases Ashok Leyland's commitment to sustainable transportation solutions and highlights their 75-year history in the industry. It also provides information on their global manufacturing presence and circular economy approach.